In [9]:
import sys,os
import numpy as np
import matplotlib.pyplot as plt

In [10]:
im_path = '/home/viha4393/Downloads/VOCdevkit/VOC2012/JPEGImages/'
anno_path = '/home/viha4393/Downloads/VOCdevkit/VOC2012/SegmentationClass/'

In [12]:
set(os.listdir(im_path)) & set(os.listdir(anno_path))

set()

In [28]:
os.path.basename(os.listdir(anno_path)[1])[:11]

'2007_000033'